# Frequency modulated continuous wave (FMCW) radar

# 1. Introduction
- Should be putting together all the concepts we've learned

# 2. Range estimation

# 3. Velocity Estimation

# 4. Range-Doppler map
- blocking N chirps to estimate velocity over

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import speed_of_light
BW = 150e6  # bandwidth
fc = 77e9  # carrier frequency
n_adc = 256  # number of adc samples
n_chirp = 512 # Number of chirps per "image"
T = 10e-6  # PRI 
PRF = 1/T
f_s = n_adc/T  # sampling frequency
dt = 1/f_s  # sampling interval
chirp_rate = BW/T
wavelen = speed_of_light/fc
fast_time = dt * np.arange(0, n_adc)
slow_time = T * np.arange(n_chirp)
n_transmitters = 1
n_receivers = 8
max_velocity = wavelen/(T*4)  # Max unambiguous velocity m/s
range_resolution = speed_of_light/(2*BW)
max_range = f_s*speed_of_light/(2*chirp_rate) 
d_rx = wavelen/2  # distance between receivers
d_tx = 4*d_rx  # distance between transmitters

# Range, velocity, and angle axes
R = range_resolution * np.arange(fast_time.size)
# R = np.arange(0, max_range, range_resolution)  # range axis
V = np.linspace(-max_velocity, max_velocity, n_chirp)  # Velocity axis
ang_ax = np.arange(-90, 90, 1)  # angle axis
print(R)



In [ ]:
print(fast_time[-1] - fast_time[0])
print(slow_time[1])


In [ ]:
# Setup the antennas
tx_loc = np.zeros((n_transmitters, 3))
tx_loc[:, 0] = d_tx * np.arange(n_transmitters)

rx_loc = np.zeros((n_receivers, 3))
rx_loc[:, 0] = tx_loc[-1][0] + d_tx + d_rx * np.arange(n_receivers)

fig = plt.figure()
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})
ax.scatter(tx_loc[:, 0], tx_loc[:, 1], tx_loc[:, 2], color='b', marker='o')
ax.scatter(rx_loc[:, 0], rx_loc[:, 1], rx_loc[:, 2], color='r', marker='x')
plt.figure()
plt.plot(tx_loc[:, 0], tx_loc[:, 1], 'bo')
plt.plot(rx_loc[:, 0], rx_loc[:, 1], 'rx')



In [ ]:
# set up targets
def create_target(pos, vel):
    _pos = np.zeros(3)
    _vel = np.zeros(3)
    _pos[0] = pos[0]
    _pos[1] = pos[1]

    _vel[0] = vel[0]
    _vel[1] = vel[1]
    return _pos + _vel[None, :] * slow_time[:, None]

targets = [
    create_target(
        pos=(50, 0),
        vel=(0, 0)
    )
]

# t2_pos = np.array([70, 0, 0])
# t2_vel = np.array([-20, 0, 0])

# t2_loc = t2_pos + t2_vel[None, :] * slow_time[:, None]

# t3_pos = np.array([70, 30, 0])
# t3_vel = np.array([0, 0, 0])
# t3_loc = t3_pos + t3_vel[None, :] * slow_time[:, None]

plt.figure()
plt.plot(tx_loc[:, 0], tx_loc[:, 1], color='b', marker='o', label='transmitters')
plt.plot(rx_loc[:, 0], rx_loc[:, 1], color='r', marker='x', label='receivers')
for tgt in targets:
    plt.plot(tgt[:, 0], tgt[:, 1], '.', label='target')
plt.legend()
plt.grid()

plt.figure()
for tgt in targets:
    plt.plot(tgt[:, 0], tgt[:, 1], '.', label='target')
plt.grid()

In [ ]:
# compute delays to targets
delays = np.zeros((n_transmitters, n_receivers, len(targets), slow_time.size))
distances = np.zeros((n_transmitters, n_receivers, len(targets), slow_time.size))
for i, tx_pos in enumerate(tx_loc):
    for j, rx_pos in enumerate(rx_loc):
        for k, tgt in enumerate(targets):
            tx_to_target = np.linalg.norm(tgt - tx_pos, axis=-1)
            rx_to_target = np.linalg.norm(tgt - rx_pos, axis=-1)
            distances[i, j, k, :] = (tx_to_target + rx_to_target)  # just for debugging 
            delays[i, j, k, :] = (tx_to_target + rx_to_target) / (speed_of_light) + 1.8e-7
delays.shape


In [ ]:
# plt.figure()
# plt.plot(delays[0, 0, :], '.')
# plt.grid()
# print(delays[0, 0, :].min())
# print(delays[0, 0, :].max())

plt.figure()
for i in range(n_receivers):
    for k in range(len(targets)):
        plt.plot(distances[0, i, k, :])
plt.figure()
for i in range(n_receivers):
    for k in range(len(targets)):
        plt.plot(delays[0, i, k, :])
# plt.ylim([0, 250])



In [29]:
# compute returned signal
sig = np.zeros((n_transmitters, n_receivers, fast_time.size, slow_time.size), dtype=np.complex128)
def received_signal(t, delay):
    # return np.exp(1.0j * (-2*np.pi * fc * delay - 2 * np.pi * chirp_rate * delay * t + np.pi *chirp_rate * delay**2))
    return np.exp(1.0j * (-2*np.pi * fc * delay - 2 * np.pi * chirp_rate * delay * t))

for i_tx in range(n_transmitters):
    for i_rx in range(n_receivers):
        for i_chirp in range(n_chirp):
            for i_tgt in range(len(targets)):
                sig[i_tx, i_rx, :, i_chirp] += received_signal(fast_time, delays[i_tx, i_rx, i_tgt, i_chirp])

In [ ]:
ref = sig[0, 0, :, :]
print(ref.shape)
range_fft = np.fft.fft(ref, axis=0)

def db(x):
    return 20 * np.log10(np.abs(x))

plt.figure()
plt.imshow(db(np.fft.fftshift(range_fft, axes=0)))

doppler_fft = np.fft.fftshift(np.fft.fft(range_fft, axis=1))
plt.figure()
plt.imshow(
    db(doppler_fft),
    extent=[V.min(), V.max(), R.min(), R.max()],
    origin='lower',
)
plt.ylabel('Range (meters)')
plt.xlabel('velocity (m/s)')

r, c = np.unravel_index(
    np.argmax(np.abs(doppler_fft).flatten()),
    doppler_fft.shape
)
rg_cut = doppler_fft[:, c]
dp_cut = doppler_fft[r, :]
plt.figure()
plt.plot(R, db(rg_cut))
plt.xlabel('Range (meters)')

plt.figure()
plt.plot(V, db(dp_cut))
plt.xlabel('velocity (m/s)')

print(V[c])
print(R[r])


In [ ]:
ref_ang = sig[0, :, :, 0].T
print(ref_ang.shape)
range_fft = np.fft.fftshift(np.fft.fft(ref_ang, axis=0), axes=0)

def db(x):
    return 20 * np.log10(np.abs(x))

plt.figure()
plt.imshow(db(range_fft))

angle_fft = np.fft.fftshift(np.fft.fft(range_fft, axis=1), axes=1)
plt.figure()
plt.imshow(
    db(angle_fft),
    aspect='auto'
)

r, c = np.unravel_index(
    np.argmax(np.abs(range_fft).flatten()),
    range_fft.shape
)
az_cut = range_fft[:, c]
plt.figure()
plt.plot(np.unwrap(np.angle(az_cut)))

# 5. Range-Angle map (our image!)
- Angle of arrival estimation over our array!